In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import os

import data as dat
from model import Model

import colours as c

dir = os.getcwd()
dir_main = os.path.dirname(os.path.abspath(dir))

In [ ]:
data = dat.Data()
scorer = dat.Scorer()
m = Model(data.smiles_vocabulary, dropout=False)

In [ ]:
with tf.Session() as session:
    
    # Set to "scorer" or "data";
    # validator should not be used in scoring.
    d = scorer
    
    # Manually evaluate compound
    # Be mindful of escape "\" characters in SMILES
    compound_smiles = "OC1=CC=CC2=C(O)C=CC=C12"
    
    literature_labels = None
    
    saver = tf.train.Saver()
    saver.restore(session, dir_main + "/trained/Solanaceae-L")
    
    print("Model restored.")
    
    print(c.GREEN + c.BOLD + c.UNDERLINE + "\nAnalysis of compound {}".format(compound_smiles) + c.END)
    
    feed_smiles = [d.indexSmiles(compound_smiles, padding=400)]
    
    predicted = session.run([m.output], feed_dict={m.batch_size: 1, m.input: feed_smiles})
    rounded = np.round(predicted[0][0])
    
    print(c.BLUE + c.BOLD + "\nPredicted values:" + c.END)
    print("Logistics: {}".format(predicted[0][0]))
    print("Rounded: {}".format(rounded))
    print("String labels: {}".format(d.labelsToReadable(rounded)))
    
    if compound_smiles in d.compounds:
        string_labels = d.compounds[compound_smiles]["labels"]
        negative_labels = d.compounds[compound_smiles]["negative"]
        literature_labels, _ = d.createLabels(string_labels, negative_labels, map_neutrals=True)
        
        literature_labels = list(literature_labels)
        
        print(c.RED + c.BOLD + "\nLiterature database:" + c.END)
        print("Expect: {}".format(literature_labels))
        print("Tags: {}".format(string_labels))
        
    else:
        print(c.RED + c.BOLD + "\nCompound not in database." + c.END)
    
    print(c.BOLD + "\nAvailable classes:" + c.END)
    print("Current: {}".format(d.classes))
    print("Training: {}".format(data.classes))
    print("Scorer: {}".format(scorer.classes))
    
    figure = plt.figure(figsize=(20, 5))
    plt.bar(d.classes, predicted[0][0], color="blue")
    plt.ylim([0, 1])
    
    if literature_labels != None:
        figure = plt.figure(figsize=(20, 5))
        plt.bar(d.classes, literature_labels, color="red")
        plt.ylim([0, 1])